<h1 style="text-align: center;">Deep Learning for Computer Vision and NLP 2025
</h1>
<h2 style="text-align: center;">Final Project</h2>
<h2 style="text-align: center;">Головна мета фінального проєкту курсу - передбачити швидкість адопції домашніх тварин на основі їхніх профілів на PetFinder.my.</h2>
<h2 style="text-align: center;">робота з базою даних PetFinder.my, найбільшого онлайн-сервісу з усиновлення тварин у Малайзії</h2>


🎢 Ваша задача полягає в розробці ефективної прогнозної моделі, яка зможе обробляти велику кількість вхідних ознак. Ви маєте показати свою здатність працювати з текстовими даними і зображеннями.



Description
link
keyboard_arrow_up
Прогнози вашої моделі для валідаційного набору даних оцінюватимуться за допомогою метрики “quadratic weighted kappa”.

Процес виставлення балів:

Максимальний бал (100) присвоюється, якщо студент набирає 0.5 або більше на метриці змагання.
Якщо студент набирає менше 0.5, бали розраховуються пропорційно.
Формула для розрахунку балів, якщо результат нижче 0.5:
*Бал = (Результат студента × 100 / 50) *

Також, кожен студент, який опублікує код свого розв’язання завдання з детальним описом кроків виконання на форумі на сторінці змагань протягом двох днів із дати закінчення змагань, отримає додаткові 10 балів, але не понад закладених максимальних 100 балів (тобто якщо за виконання цієї умови студент отримує понад 100 б., оцінка роботи складатиме 100 б.).

Щоб розрахунок метрики був коректним, вам потрібно сформувати файл .csv з ідентифікатором тварини та прогнозованою швидкістю адопції. Порядок рядків не має значення. Файл повинен мати заголовок і виглядати так:

PetID,AdoptionSpeed
378fcc4fc,3
73c10e136,2
72000c4c5,1
e147a4b9f,4
etc..

Ретельно вивчіть дані: проаналізуйте структуру та особливості наданих зображень і текстів.

Почніть з простого: створіть базову модель і поступово її вдосконалюйте.

Експериментуйте з різними архітектурами: спробуйте різні моделі для обробки зображень (CNN, ResNet) та тексту (LSTM, Transformers).

Використовуйте попередньо навчені моделі: застосуйте transfer learning для покращення результатів.
Приділіть увагу попередній обробці даних: очистіть текст, нормалізуйте зображення.

Регулярно перевіряйте продуктивність моделі на валідаційному наборі, використовуючи офіційну метрику змагання — “quadratic weighted kappa”.

Додайте регуляризацію до функції втрат, щоб запобігти перенавчанню.

Вивчайте рішення учасників аналогічних змагань: аналізуйте публічні ноутбуки для нових ідей.

Оптимізуйте гіперпараметри, наприклад, з використанням фреймворку Optuna.

Зверніть увагу на дисбаланс класів — застосуйте відповідні методи, якщо потрібно.

Не забувайте про інтерпретацію моделі: спробуйте зрозуміти, які фактори найбільше впливають на прогноз.

Перед отриманням прогнозів для валідаційного набору (пере-) навчіть вашу фінальну модель на всіх доступних даних (тренувальна + тестова вибірки).

Сформуйте файл .csv з ідентифікатором тварини та прогнозованою швидкістю адопції. Порядок рядків не має значення.

Після надання файлу з прогнозами та розрахунку метрики змагання подумайте про можливість покращення результатів і продовжуйте експериментувати над підвищенням метрики.

Dataset Description
Набір даних для дослідження і навчання моделі містить 6 432 об'єкти.

Вхідні дані даної задачі мають наступні характеристики:

Зображення:
Фотографії тварин (собак та котів).
Можуть бути як одиничні зображення, так і декілька зображень для однієї тварини.
Текстові дані:
Опис тварини: текстова інформація, яка описує характер, історію та особливості тварини.
Можливо, включає інформацію про породу, вік, стан здоров'я тощо.
Цільова змінна:
Категорія швидкості усиновлення (наприклад, від 1 до 4, де 1 — найшвидше усиновлення, 4 — найповільніше)
Ваше завдання — використати ці дані для створення моделі, яка прогнозуватиме швидкість усиновлення тварини.

Для роботи вам знадобляться наступні файли, які ви знайдете на сторінці змагання:

train.csv - навчальний набір
test.csv - валідаційний набір
sample_submission.csv - файл-зразок для подання прогнозів у правильному форматі

## **Import the required libraries.**

In [1]:
!pip install seaborn

In [2]:
import glob
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import cohen_kappa_score

from sklearn.compose import ColumnTransformer

from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
from torchvision import models

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

import warnings
# filter warnings
warnings.filterwarnings('ignore')


## 1. **Підготовка датасету для текстів:**

## **Data loading:**

In [3]:
DATA_DIR = "./PetFinderDatasets"

train_path = os.path.join(DATA_DIR, "train.csv")
test_path  = os.path.join(DATA_DIR, "test.csv")
sample_sub = os.path.join(DATA_DIR, "sample_submission.csv")

train = pd.read_csv(train_path)
test  = pd.read_csv(test_path)
sample = pd.read_csv(sample_sub)

print("Train shape:", train.shape)
display(train.head())
print("Test  shape:", test.shape)
display(test.head())
print("Sample head:")
display(sample.head())

Train shape: (6431, 3)


,PetID,Description,AdoptionSpeed
0,d3b4f29f8,Mayleen and Flo are two lovely adorable sister...,2
1,e9dc82251,A total of 5 beautiful Tabbys available for ad...,2
2,8111f6d4a,Two-and-a-half month old girl. Very manja and ...,2
3,693a90fda,Neil is a healthy and active ~2-month-old fema...,2
4,9d08c85ef,Gray kitten available for adoption in sungai p...,2


Test  shape: (1891, 2)


,PetID,Description
0,6697a7f62,This cute little puppy is looking for a loving...
1,23b64fe21,These 3 puppies was rescued from a mechanic sh...
2,41e824cbe,"Ara needs a forever home! Believe me, he's a r..."
3,6c3d7237b,i rescue this homeless dog 2 years ago but my ...
4,97b0b5d92,We found him at a shopping mall at a very clea...


Sample head:


,PetID,AdoptionSpeed
0,6697a7f62,1
1,23b64fe21,2
2,41e824cbe,3
3,6c3d7237b,4


## **Data checking:**

In [4]:
# базова перевірка потрібних колонок
need_train_cols = ["PetID", "AdoptionSpeed", "Description"]
need_test_cols  = ["PetID", "Description"]

for c in need_train_cols:
    assert c in train.columns, f"В train.csv не знайдено колонку {c}"
for c in need_test_cols:
    assert c in test.columns, f"В test.csv не знайдено колонку {c}"

train["Description"] = train["Description"].fillna("") # замінюємо NaN на пустий рядок, щоб не було збоїв у TF-IDF.
print("Приклади описів у train:")
print(train["Description"].head(5))

test["Description"]  = test["Description"].fillna("")

print("\nClass distribution (AdoptionSpeed) in train:")
display(train["AdoptionSpeed"].value_counts().sort_index())

Приклади описів у train:
0    Mayleen and Flo are two lovely adorable sister...
1    A total of 5 beautiful Tabbys available for ad...
2    Two-and-a-half month old girl. Very manja and ...
3    Neil is a healthy and active ~2-month-old fema...
4    Gray kitten available for adoption in sungai p...
Name: Description, dtype: object

Class distribution (AdoptionSpeed) in train:


AdoptionSpeed
1    1197
2    1773
3    1328
4    2133
Name: count, dtype: int64

## **Data preparation:**

Train/Validation split

In [5]:
X_tr, X_val, y_tr, y_val = train_test_split(
    train["Description"],
    train["AdoptionSpeed"],
    test_size=0.2,
    random_state=42,
    stratify=train["AdoptionSpeed"]
)

print("Train split size:", X_tr.shape[0])
print("Val split size:  ", X_val.shape[0])
print("Target distrib train:")
print(y_tr.value_counts(normalize=True).sort_index())
print("Target distrib val:")
print(y_val.value_counts(normalize=True).sort_index())

Train split size: 5144
Val split size:   1287
Target distrib train:
AdoptionSpeed
1    0.186236
2    0.275661
3    0.206454
4    0.331649
Name: proportion, dtype: float64
Target distrib val:
AdoptionSpeed
1    0.185703
2    0.275835
3    0.206682
4    0.331779
Name: proportion, dtype: float64


## **Побудова пайплайну TF-IDF + LogisticRegression:**

In [6]:
ct = ColumnTransformer([
    ("w", TfidfVectorizer(max_features=10000, ngram_range=(1,2),
                          min_df=3, sublinear_tf=True,
                          lowercase=True, strip_accents="unicode"), "Description"),
    ("c", TfidfVectorizer(analyzer="char", ngram_range=(3,5),
                          min_df=3, sublinear_tf=True), "Description")
])

from sklearn.pipeline import Pipeline
pipe_char = Pipeline([
    ("feats", ct),
    ("clf", LogisticRegression(max_iter=300, C=1.0, solver="lbfgs",
                               multi_class="multinomial", class_weight=None,
                               random_state=42))
])

pipe_char.fit(train[["Description"]], train["AdoptionSpeed"])
pred = pipe_char.predict(pd.DataFrame({"Description": X_val}))
print("QWK (word+char):", cohen_kappa_score(y_val, pred, weights="quadratic"))

QWK (word+char): 0.8324351768853542


## **Навчання і локальна перевірка (QWK):**

## **Фінальне навчання на всьому train + submission.csv:**

In [7]:
pipe_char.fit(train[["Description"]], train["AdoptionSpeed"])
preds = pipe_char.predict(test[["Description"]])
submission = pd.DataFrame({"PetID": test["PetID"], "AdoptionSpeed": preds})
submission.to_csv("submission_text_only.csv", index=False)

## 2. **Підготовка датасету для картинок:**

In [8]:
from torchvision.transforms.autoaugment import AutoAugment, AutoAugmentPolicy
# трансформації для зображень
img_tfms_train = transforms.Compose([
    transforms.RandomResizedCrop((224, 224), scale=(0.7, 1.0), ratio=(0.75, 1.333)),
    AutoAugment(policy=AutoAugmentPolicy.IMAGENET),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
img_tfms_eval = transforms.Compose([
    transforms.Resize((240, 240)),
    transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [9]:
# === Utility: find image path for given PetID (search recursively) ===
def find_image(img_dir, petid):
    # search in root and subfolders, any extension
    pats = [
        os.path.join(img_dir, f"{petid}*.*"),
        os.path.join(img_dir, "**", f"{petid}*.*"),
    ]
    for pat in pats:
        hits = glob.glob(pat, recursive=True)
        hits = [h for h in hits if h.lower().endswith((".jpg", ".jpeg", ".png"))]
        if hits:
            return sorted(hits)[0]
    print("None")
    return None

In [10]:
train.head()

,PetID,Description,AdoptionSpeed
0,d3b4f29f8,Mayleen and Flo are two lovely adorable sister...,2
1,e9dc82251,A total of 5 beautiful Tabbys available for ad...,2
2,8111f6d4a,Two-and-a-half month old girl. Very manja and ...,2
3,693a90fda,Neil is a healthy and active ~2-month-old fema...,2
4,9d08c85ef,Gray kitten available for adoption in sungai p...,2


In [11]:
train.shape


(6431, 3)

In [12]:
# Pre-compute image paths

TRAIN_IMG_DIR = 'PetFinderDatasets/images/images/train'
TEST_IMG_DIR = 'PetFinderDatasets/images/images/test'

test["img_path"]  = test["PetID"].apply(lambda x: find_image(TEST_IMG_DIR,  x))

train["img_path"] = train["PetID"].apply(lambda x: find_image(TRAIN_IMG_DIR, x))
print("Missing train images:", train["img_path"].isna().sum())

print("Missing test  images:", test["img_path"].isna().sum())
# TEST: keep ALL rows (submission must include every PetID)
# We'll use a placeholder tensor when image is missing/corrupted.
placeholder = torch.zeros(3, 224, 224)  # black square tensor

None
None
None
None
Missing train images: 0
Missing test  images: 4


In [13]:
# map labels {1,2,3,4} → indices {0,1,2,3}
classes_sorted = sorted(train["AdoptionSpeed"].unique())  # e.g. [1,2,3,4]
label2idx = {lab: i for i, lab in enumerate(classes_sorted)}
idx2label = {i: lab for i, lab in enumerate(classes_sorted)}
num_classes = len(classes_sorted)

In [14]:

class PetDataset(Dataset):
    def __init__(self, df, training=True, tfms_train=None, tfms_eval=None):
        self.df = df.reset_index(drop=True)
        self.training = training
        self.tfms_train = tfms_train
        self.tfms_eval  = tfms_eval
        self.has_labels = "AdoptionSpeed" in self.df.columns

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
        row = self.df.iloc[i]
        img_path = row["img_path"]
        
        try:
            img = Image.open(img_path).convert("RGB")
            tfm = self.tfms_train if self.training else self.tfms_eval
            x = tfm(img)
        except:
            x = placeholder.clone()

        if self.has_labels:
            # for val and train
            y = label2idx[int(row["AdoptionSpeed"])]
            return x, y
        else:
            # for test
            return x, row["PetID"]
            

## **Train/Val split + DataLoader-и:**

In [15]:
# train/val split
tr_df, val_df = train_test_split(
    train, test_size=0.2, random_state=42, stratify=train["AdoptionSpeed"]
)

# створюємо датасети
tr_ds  = PetDataset(tr_df,  training=True,  tfms_train=img_tfms_train, tfms_eval=img_tfms_eval)
val_ds = PetDataset(val_df,  training=False, tfms_train=img_tfms_train, tfms_eval=img_tfms_eval)  # eval tfms
te_ds  = PetDataset(test,    training=False, tfms_train=img_tfms_train, tfms_eval=img_tfms_eval)  # eval tfms, returns (x, PetID)

# DataLoader-и
tr_dl  = DataLoader(tr_ds, batch_size=32, shuffle=True,  num_workers=2, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)
te_dl  = DataLoader(te_ds,  batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

print("Train batches:", len(tr_dl))
print("Val batches:  ", len(val_dl))
print("Test batches: ", len(te_dl))

Train batches: 161
Val batches:   41
Test batches:  60


In [16]:
for images, labels in tr_dl:
    print(images.shape)   # (batch_size, 3, 224, 224)
    print(labels[:5])
    break

torch.Size([32, 3, 224, 224])
tensor([2, 0, 1, 1, 0])


In [17]:
for images, labels in val_dl:
    print(images.shape)   # (batch_size, 3, 224, 224)
    print(labels[:5])
    break

torch.Size([32, 3, 224, 224])
tensor([2, 0, 3, 2, 2])


## **Модель ResNet18 (Transfer Learning):**

In [18]:
# створюємо ResNet18 з pretrained вагами
model = models.resnet18(pretrained=True)

# замінюємо останній шар (fc) на 4 класи
in_feats = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(p=0.25), 
    nn.Linear(in_feats, num_classes)
)
print(model.fc)

Sequential(
  (0): Dropout(p=0.25, inplace=False)
  (1): Linear(in_features=512, out_features=4, bias=True)
)


## **Loss + Optimizer + GPU:**

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

model = model.to(device)

# функція втрат (класифікація)
criterion = nn.CrossEntropyLoss()

# оптимізатор (Adam)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

Device: cuda


## **Цикл навчання (train/val) + збереження best-моделі:**

In [21]:
#from tqdm.auto import tqdm # прогресс-бар

epochs = 50   # baseline, можна збільшити
best_val_acc = 0.0
history = {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}

model.to(device)

for epoch in range(1, epochs+1):
    # ----- TRAIN -----
    model.train()
    train_loss, correct, total = 0.0, 0, 0

    #train_loader = tqdm(tr_dl, desc=f"Epoch {epoch:02d} [Train]")

    for xb, yb in tr_dl:
        xb, yb = xb.to(device), yb.to(device)
        
        optimizer.zero_grad()
        outputs = model(xb)
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * xb.size(0)
        preds = outputs.argmax(1)
        correct += (preds == yb).sum().item()
        total += yb.size(0)

        #train_loader.set_postfix(loss=f"{(train_loss/total):.4f}", acc=f"{(correct/total):.3f}")
    
    train_loss /= total
    train_acc = correct / total
    
    # ----- VALIDATION -----
    model.eval()

    val_loss, correct, total = 0.0, 0, 0
    #val_loader = tqdm(val_dl, desc=f"Epoch {epoch:02d} [Val]")

    with torch.no_grad():
        for xb, yb in val_dl:
            
            xb, yb = xb.to(device), yb.to(device)        
 
            outputs = model(xb)
            loss = criterion(outputs, yb)
            
            val_loss += loss.item() * xb.size(0)
            preds = outputs.argmax(1)
            correct += (preds == yb).sum().item()
            total += yb.size(0)

            #val_loader.set_postfix(loss=f"{(val_loss/total):.4f}", acc=f"{(correct/total):.3f}")

    val_loss /= total
    val_acc = correct / total
    
    # збереження best
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_resnet18.pth")
    
    history["train_loss"].append(train_loss)
    history["val_loss"].append(val_loss)
    history["train_acc"].append(train_acc)
    history["val_acc"].append(val_acc)
    
    print(f"Epoch {epoch:02d} | Train loss {train_loss:.4f} acc {train_acc:.3f} "
          f"| Val loss {val_loss:.4f} acc {val_acc:.3f}")

print("✅ Training finished. Best val acc:", best_val_acc)

Epoch 01 | Train loss 1.3915 acc 0.324 | Val loss 1.4104 acc 0.353
Epoch 02 | Train loss 1.3877 acc 0.324 | Val loss 1.3517 acc 0.334
Epoch 03 | Train loss 1.3691 acc 0.325 | Val loss 1.3927 acc 0.274
Epoch 04 | Train loss 1.3659 acc 0.329 | Val loss 1.7757 acc 0.322
Epoch 05 | Train loss 1.3747 acc 0.321 | Val loss 1.3515 acc 0.340
Epoch 06 | Train loss 1.3619 acc 0.332 | Val loss 1.3850 acc 0.318
Epoch 07 | Train loss 1.3682 acc 0.335 | Val loss 1.7715 acc 0.330
Epoch 08 | Train loss 1.3690 acc 0.326 | Val loss 1.3541 acc 0.342
Epoch 09 | Train loss 1.3584 acc 0.343 | Val loss 1.4055 acc 0.350
Epoch 10 | Train loss 1.3589 acc 0.337 | Val loss 1.3780 acc 0.338
Epoch 11 | Train loss 1.3582 acc 0.337 | Val loss 1.3692 acc 0.329
Epoch 12 | Train loss 1.3541 acc 0.349 | Val loss 1.3586 acc 0.336
Epoch 13 | Train loss 1.3566 acc 0.334 | Val loss 1.3636 acc 0.343
Epoch 14 | Train loss 1.3558 acc 0.343 | Val loss 1.3486 acc 0.345
Epoch 15 | Train loss 1.3559 acc 0.343 | Val loss 1.3583 acc 0

## **Інференс на test + submission.csv:**

In [22]:
# підвантажимо best-ваги, якщо збережені на кроці 5
if os.path.exists("best_resnet18.pth"):
    model.load_state_dict(torch.load("best_resnet18.pth", map_location=device))
    print("Loaded best weights: best_resnet18.pth")
model.eval()

# мапа індексів у вихідні ярлики (підлаштовуємось під ваш train: чи це [0..3], чи [1..4])
classes_sorted = sorted(train["AdoptionSpeed"].unique())   # напр., [1,2,3,4] або [0,1,2,3]
idx2label = {i: classes_sorted[i] for i in range(4)}

preds, ids = [], []
with torch.no_grad():
    for xb, pet_ids in te_dl:
        xb = xb.to(device)
        out = model(xb)
        pred_idx = out.argmax(1).cpu().numpy()
        pred_lab = [idx2label[int(i)] for i in pred_idx]
        preds.extend(pred_lab)
        ids.extend(pet_ids)

submission = pd.DataFrame({"PetID": ids, "AdoptionSpeed": np.array(preds, dtype=int)})
submission.to_csv("submission_images_only.csv", index=False)
print("✅ Saved:", "submission_images_only.csv")
display(submission.head())

Loaded best weights: best_resnet18.pth
✅ Saved: submission_images_only.csv


,PetID,AdoptionSpeed
0,6697a7f62,4
1,23b64fe21,2
2,41e824cbe,4
3,6c3d7237b,4
4,97b0b5d92,4


## 3. **Ансамбль (текст + зображення):**

In [45]:
# Копія тестових даних і підготовка PetID
test = test.copy()
test["PetID"] = test["PetID"].astype(str).str.strip()
X_test_desc = test[["Description"]].fillna("")

*Розразовуємо якість ансамблю.*

In [46]:
qwk_text = cohen_kappa_score(y_val, pred_text_val, weights="quadratic")
print("QWK text:", qwk_text)

# 2) Image probabilities on validation
proba_img_list, ids_val = [], []
model.eval()
with torch.no_grad():
    for xb, pet_ids in val_dl:
        xb = xb.to(device)
        out = model(xb)
        probs = torch.softmax(out, dim=1).cpu().numpy()
        proba_img_list.append(probs)
        ids_val.extend(pet_ids)

proba_img_val = np.vstack(proba_img_list)
pred_img_val = proba_img_val.argmax(axis=1)

qwk_img = cohen_kappa_score(y_val, pred_img_val, weights="quadratic")
print("QWK img:", qwk_img)

# 3) Ensemble sweep over alpha
alphas = np.linspace(0.0, 1.0, 11)
best_alpha, best_qwk = None, -1
for a in alphas:
    p = a * proba_text_val + (1.0 - a) * proba_img_val
    pred = p.argmax(axis=1)
    q = cohen_kappa_score(y_val, pred, weights="quadratic")
    if q > best_qwk:
        best_qwk, best_alpha = q, a

print(f"Best ensemble QWK: {best_qwk:.5f} at alpha={best_alpha:.2f}")

QWK text: 0.6348906518987774
QWK img: 0.17340372675825344
Best ensemble QWK: 0.63625 at alpha=0.80


*Готуємо сабмішин*

In [48]:
# Ймовірності з текстової моделі
proba_text = pipe_char.predict_proba(X_test_desc)
classes_text = pipe_char.named_steps["clf"].classes_
text_cols = [f"text_{c}" for c in classes_text]
proba_text_df = pd.DataFrame(proba_text, columns=text_cols)
proba_text_df["PetID"] = test["PetID"].values

In [49]:
# Перевірка, чи можна використати ймовірності зображень
use_images = ("proba_img" in locals()) and ("ids" in locals()) and (proba_img is not None)
if use_images:
    classes_img = sorted(train["AdoptionSpeed"].unique().tolist())
    if getattr(proba_img, "shape", None) is None or proba_img.shape[0] == 0:
        use_images = False
    elif proba_img.shape[1] != len(classes_img):
        if proba_img.shape[1] == len(classes_text):
            classes_img = list(classes_text)
        else:
            use_images = False

In [50]:
# Формуємо DataFrame для зображень (якщо доступні)
if use_images:
    img_cols = [f"img_{c}" for c in classes_img]
    proba_img_df = pd.DataFrame(proba_img, columns=img_cols)
    proba_img_df["PetID"] = pd.Series(ids).astype(str).str.strip()
else:
    proba_img_df = pd.DataFrame({"PetID": []})

In [51]:
# Повний список міток класів (текст + зображення)
labels_all = sorted(set(classes_text).union(set(classes_img))) if use_images else list(classes_text)

In [52]:
# Додаємо відсутні стовпці у текстовому/зображенневому DataFrame
for c in labels_all:
    tc = f"text_{c}"
    if tc not in proba_text_df.columns:
        proba_text_df[tc] = 0.0
if use_images:
    for c in labels_all:
        ic = f"img_{c}"
        if ic not in proba_img_df.columns:
            proba_img_df[ic] = np.nan

# З’єднуємо тест з текстовими ймовірностями
keep_text = ["PetID"] + [f"text_{c}" for c in labels_all]
merged = test[["PetID"]].merge(proba_text_df[keep_text], on="PetID", how="left")

# Якщо доступні зображення — додаємо їх
if use_images:
    keep_img = ["PetID"] + [f"img_{c}" for c in labels_all]
    merged = merged.merge(proba_img_df[keep_img], on="PetID", how="left")

In [53]:
# Вага тексту і зображень
alpha, beta = 0.5, 0.5
if use_images:
    img_cols_all = [f"img_{c}" for c in labels_all]
    has_img = merged[img_cols_all].notna().all(axis=1).values
    w_text = np.where(has_img, alpha, 1.0)   # якщо є зображення
    w_img  = np.where(has_img, beta,  0.0)   # якщо немає зображення
else:
    w_text = np.ones(len(merged), dtype=float)
    w_img  = np.zeros(len(merged), dtype=float)

In [54]:
# Обчислення ансамблевих ймовірностей
for c in labels_all:
    text_col = merged[f"text_{c}"].fillna(0.0).to_numpy()
    img_col  = merged[f"img_{c}"].fillna(0.0).to_numpy() if f"img_{c}" in merged.columns else np.zeros(len(merged))
    merged[f"p_{c}"] = w_text * text_col + w_img * img_col

# Нормалізація ймовірностей (щоб у рядку сума = 1)
p_cols = [f"p_{c}" for c in labels_all]
row_sum = merged[p_cols].sum(axis=1).replace(0, 1.0)
merged[p_cols] = merged[p_cols].div(row_sum, axis=0)

# Фінальний клас = argmax по ансамблю
pos = merged[p_cols].values.argmax(axis=1)
labels_by_pos = np.array(labels_all)
merged["AdoptionSpeed"] = labels_by_pos[pos].astype(int)

In [55]:
# Формуємо сабмішн у форматі Kaggle
submission = test[["PetID"]].merge(merged[["PetID","AdoptionSpeed"]], on="PetID", how="left")
submission.to_csv("submission_ensemble_text_img.csv", index=False)
print("✅ Saved:", "submission_ensemble_text_img.csv")

✅ Saved: submission_ensemble_text_img.csv



---

# Звіт по проєкту Pet Adoption Prediction

## Мета

Розробити модель прогнозування швидкості усиновлення тварин на основі описів (текст) та фотографій (зображення). Використовувались:

* **Тексти:** опис тварин з набору даних PetFinder.my.
* **Фото:** обробка з використанням ResNet18.
* **Ансамбль:** поєднання ймовірностей від текстової та зображеневої моделей.

## Поточні результати

* **Текстова модель (LogisticRegression + TF-IDF):** показала стабільний базовий результат, особливо завдяки поєднанню word- та char-n-gram.
* **Зображення (ResNet18):** модель працює, але якість виявилась нижчою очікуваної через недопрацювання у препроцесінгу фото (нерівномірні розміри, відсутність нормалізації, слабке використання даних).
* **Ансамбль (text+image):** очікуваного приросту на реальному тесті не дало, якість залишилася на рівні текстової моделі.

## Проблеми

1. **Фото**

   * Недостатня очистка та нормалізація даних.
   * Використання однієї моделі (ResNet18) без тонкого тюнінгу чи data augmentation.
   * Ігнорування кількості зображень (частина тварин має 1 фото, інші — до 5).

2. **Тексти**

   * Використовувалась проста TF-IDF + Logistic Regression.
   * Не застосовували трансформери (BERT, mBERT, distilBERT) чи сучасні ембеддинги.
   * Слабке врахування метаданих (наприклад, довжина опису, кількість символів).

3. **Ансамбль**

   * Фіксовані ваги (0.5/0.5) для тексту та фото.
   * Відсутність адаптивного підбору ваг (наприклад, залежно від кількості фото).
   * Не використано мета-модель для комбінування результатів.

## План покращення

### 1. Зображення

* Використати **ResNet50 / EfficientNet / ViT** з попередньо натренованими вагами.
* Додати **data augmentation** (кропи, фліпи, кольорові трансформації).
* Нормалізувати фото під стандартні mean/std ImageNet.
* Використати **кількість фото як окрему ознаку** у табличній моделі.
* Спробувати **feature extraction** (витягнути embeddings з CNN і передати в ансамблеву модель разом із текстовими ознаками).

### 2. Тексти

* Перейти від TF-IDF до **BERT-подібних моделей** для створення sentence embeddings.
* Додати прості ознаки: довжина опису, кількість слів, кількість символів.
* Спробувати LightGBM / CatBoost поверх TF-IDF + метаданих.

### 3. Ансамбль

* Замість фіксованих ваг використовувати:

  * **Мета-модель (stacking):** логістична регресія чи градієнтний бустинг, яка вчиться поєднувати `proba_text` + `proba_img`.
  * **Адаптивні ваги:** залежно від кількості фото або впевненості моделі.
* Можна додати простий “confidence gating”: якщо фото-модель впевнена (>0.8), дати їй більшу вагу.

### 4. Інфраструктура та експерименти

* Вести **чіткий трекінг експериментів** (MLflow, Weights\&Biases).
* Зробити **валідацію з розбиттям по PetID** (щоб фото однієї тварини не потрапляли і в train, і в val).
* Використати **stratified K-fold cross-validation** для стабільних оцінок.

## Висновки

* Текстова модель показала сильний baseline.
* Якість на реальному тесті обмежена слабкою обробкою зображень.
* Основний потенціал для покращення — **поглиблена робота з фото**, а також перехід до більш потужних текстових моделей.
* У майбутніх ітераціях варто розглядати ансамблі, які поєднують CNN-ембеддинги з трансформерними текстовими ембеддингами і додатковими табличними ознаками (кількість фото, довжина опису, вік/порода).

---